## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210225.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,662310,36300,144200,842810,688093,0.816427,221557,2021-02-24
1,Aragón,112635,6300,22600,141535,120420,0.850814,44734,2021-02-24
2,Asturias,112055,6300,17400,135755,109192,0.804331,38716,2021-02-24
3,Baleares,56520,3200,19900,79620,55834,0.701256,20975,2021-02-24
4,Canarias,129840,7000,37100,173940,133624,0.768219,51123,2021-02-24
5,Cantabria,55725,2900,9900,68525,53706,0.783743,20018,2021-02-24
6,Castilla y Leon,246585,13500,40800,300885,236901,0.787347,88071,2021-02-24
7,Castilla La Mancha,167175,9200,34800,211175,154252,0.730446,65766,2021-02-24
8,Cataluña,570410,31400,132700,734510,572114,0.778906,184380,2021-02-24
9,C. Valenciana,288010,15700,86200,389910,301372,0.772927,100793,2021-02-24


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,662310,36300,144200,842810,688093,0.816427,221557,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,112635,6300,22600,141535,120420,0.850814,44734,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,112055,6300,17400,135755,109192,0.804331,38716,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,56520,3200,19900,79620,55834,0.701256,20975,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,129840,7000,37100,173940,133624,0.768219,51123,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,55725,2900,9900,68525,53706,0.783743,20018,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,246585,13500,40800,300885,236901,0.787347,88071,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,167175,9200,34800,211175,154252,0.730446,65766,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,570410,31400,132700,734510,572114,0.778906,184380,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,288010,15700,86200,389910,301372,0.772927,100793,24/2/21,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [14]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [15]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,25/2/21,Andalucía,662310,36300,144200,842810,688093,0.816427,221557,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,25/2/21,Aragón,112635,6300,22600,141535,120420,0.850814,44734,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,25/2/21,Asturias,112055,6300,17400,135755,109192,0.804331,38716,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,25/2/21,Baleares,56520,3200,19900,79620,55834,0.701256,20975,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,25/2/21,Canarias,129840,7000,37100,173940,133624,0.768219,51123,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,25/2/21,Cantabria,55725,2900,9900,68525,53706,0.783743,20018,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,25/2/21,Castilla y Leon,246585,13500,40800,300885,236901,0.787347,88071,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,25/2/21,Castilla La Mancha,167175,9200,34800,211175,154252,0.730446,65766,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,25/2/21,Cataluña,570410,31400,132700,734510,572114,0.778906,184380,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,25/2/21,C. Valenciana,288010,15700,86200,389910,301372,0.772927,100793,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [16]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [17]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [19]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,24/2/21,Navarra,55725.0,2900.0,11200.0,69825,53145,0.761117,20422.0,NaN,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
736,24/2/21,País Vasco,148410.0,7900.0,37900.0,194210,117427,0.604639,45475.0,NaN,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
737,24/2/21,Ceuta,4320.0,400.0,1500.0,6220,5705,0.917203,1087.0,NaN,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
738,24/2/21,Melilla,3150.0,400.0,1500.0,5050,4334,0.858218,1266.0,NaN,23/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [20]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [21]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [22]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [23]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,25/2/21,Navarra,55725.0,2900.0,11200.0,69825,55427,0.793799,20842.0,NaN,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
756,25/2/21,País Vasco,148410.0,7900.0,37900.0,194210,119747,0.616585,45667.0,NaN,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
757,25/2/21,Ceuta,4320.0,400.0,1500.0,6220,5956,0.957556,1090.0,NaN,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
758,25/2/21,Melilla,3150.0,400.0,1500.0,5050,4557,0.902376,1266.0,NaN,24/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [24]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [25]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)